In [3]:
#추천시스템
# https://github.com/wikibook/ml-definitive-guide

# 확률적 경사 하강법을 이용한 행렬분해

In [4]:
import numpy as np

In [5]:
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])

In [6]:
#분해 행렬 P&Q 초기화
num_users, num_items = R.shape

In [7]:
#col
num_users

4

In [8]:
#row
num_items

5

In [9]:
#잠재요인차원
k=3

In [10]:
#P와Q 행렬의 크기 지정, 정규 분포를 가진 임의 값으로 입력
np.random.seed(1)
P = np.random.normal(scale=1./k, size = (num_users,k))
Q = np.random.normal(scale=1./k, size = (num_items,k))

In [11]:
#행렬값의 위치 인덱스를 추출해 이 인덱스에 있는 실제 R 행렬값과 분해된 P,Q 를 이용해 다시 조합된 예측 행렬값의 RMSE 반환
from sklearn.metrics import mean_squared_error

In [12]:
def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P,Q.T) #행렬 P와 Q의 내적으로 예측 R 생성
    
    #위치 인덱스를 추출 -> 실제 R행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [13]:
#SGD 기반 행렬 분해 
#steps = SGD 반복해서 업데이트할 횟수 
#learning_rate = SGD 학습률
#r_lambda = L2 regularization 계수
steps = 1000
learning_rate = 0.01
r_lambda = 0.01


#R>0인 행 위치, 열위치 값을 non_zeros 리스트에 저장
non_zeros = [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]

In [14]:
#SGD 기법으로 P와Q 매트릭스 업데이트 ->50 회 반복 할때마다 오류값 출력
for step in range(steps):
    for i,j,r in non_zeros:
        eij = r - np.dot(P[i,:], Q[j,:].T) #실제 값과 예측값의 차이인 오류 값을 구함
        P[i,:] = P[i, :] + learning_rate*(eij*Q[j,:]-r_lambda*P[i,:]) #regularization을 반영한 SGD 업데이트 공식 적용
        Q[i,:] = Q[i, :] + learning_rate*(eij*Q[j,:]-r_lambda*Q[i,:])
        
        rmse = get_rmse(R,P,Q,non_zeros)
        if(step%50) == 0:
            print("### iterarion step:", step,"rmse:",rmse)

### iterarion step: 0 rmse: 3.264818196112103
### iterarion step: 0 rmse: 3.264370499465061
### iterarion step: 0 rmse: 3.261672121442261
### iterarion step: 0 rmse: 3.2624527494419877
### iterarion step: 0 rmse: 3.2627451924379254
### iterarion step: 0 rmse: 3.2624024947875943
### iterarion step: 0 rmse: 3.2607390798068367
### iterarion step: 0 rmse: 3.2595221381444444
### iterarion step: 0 rmse: 3.260954730435898
### iterarion step: 0 rmse: 3.2609508165214223
### iterarion step: 0 rmse: 3.2607914544151466
### iterarion step: 0 rmse: 3.260595390737092
### iterarion step: 50 rmse: 1.6672341428098716
### iterarion step: 50 rmse: 1.6745479049778667
### iterarion step: 50 rmse: 1.6572330212582147
### iterarion step: 50 rmse: 1.6520681864262174
### iterarion step: 50 rmse: 1.6534861686695055
### iterarion step: 50 rmse: 1.6534469058766692
### iterarion step: 50 rmse: 1.6525286596767195
### iterarion step: 50 rmse: 1.652526781210148
### iterarion step: 50 rmse: 1.6559357535642625
### iterar

### iterarion step: 700 rmse: 1.4147716877761511
### iterarion step: 700 rmse: 1.4121022885272216
### iterarion step: 700 rmse: 1.4158526586248263
### iterarion step: 700 rmse: 1.414855817585214
### iterarion step: 700 rmse: 1.416508996859151
### iterarion step: 700 rmse: 1.412397236433255
### iterarion step: 700 rmse: 1.4146779195534003
### iterarion step: 700 rmse: 1.415487799863068
### iterarion step: 750 rmse: 1.4363085424148079
### iterarion step: 750 rmse: 1.4368491445080773
### iterarion step: 750 rmse: 1.4368793824635695
### iterarion step: 750 rmse: 1.4361007744923164
### iterarion step: 750 rmse: 1.4365116463946859
### iterarion step: 750 rmse: 1.4339120005085042
### iterarion step: 750 rmse: 1.4378128729862163
### iterarion step: 750 rmse: 1.4366152326552926
### iterarion step: 750 rmse: 1.4379370248184482
### iterarion step: 750 rmse: 1.433619871439622
### iterarion step: 750 rmse: 1.4363110354004855
### iterarion step: 750 rmse: 1.4373246573722052
### iterarion step: 800 r

In [15]:
#P*Q.T 예측 행렬 출력
pred_matrix = np.dot(P,Q.T)
print('예측 행렬:\n', np.round(pred_matrix,3))

예측 행렬:
 [[ 3.784  0.769  2.095  2.309 -0.55 ]
 [ 2.389  5.173  3.897  2.185 -0.48 ]
 [ 2.449  4.595  3.704  1.922 -0.357]
 [ 3.933  1.239  2.366  2.581 -0.647]]
